# ViTalia - NonTrained ViT 

___________________________

In [2]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
import warnings


warnings.filterwarnings("ignore")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = models.vision_transformer.vit_b_16(weights=None, num_classes=2).to(device)


data_dir = '/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/'


class MalariaDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(["Parasitized", "Uninfected"]):
            class_dir = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_file))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except (UnidentifiedImageError, FileNotFoundError):
            print(f"Error loading image {img_path}. Skipping.")
            return self.__getitem__((idx + 1) % len(self)) 

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

full_dataset = MalariaDataset(data_dir, transform=transform)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, pin_memory=True)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) 
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 15
patience = 3
best_loss = float('inf')
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")


    if avg_loss < best_loss:
        best_loss = avg_loss
        early_stop_counter = 0 
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered.")
            break


model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision * 100:.2f}%")
print(f"Test Recall: {recall * 100:.2f}%")
print(f"Test F1 Score: {f1 * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)


Epoch [1/15]:  55%|█████▍    | 755/1378 [05:19<04:25,  2.35it/s, loss=0.061] 

Error loading image /kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized/Thumbs.db. Skipping.


Epoch [1/15]:  58%|█████▊    | 804/1378 [05:40<04:01,  2.38it/s, loss=0.245] 

Error loading image /kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected/Thumbs.db. Skipping.


Epoch [1/15]: 100%|██████████| 1378/1378 [09:44<00:00,  2.36it/s, loss=0.343] 


...

Error loading image /kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected/Thumbs.db. Skipping.


Epoch [15/15]:  38%|███▊      | 524/1378 [03:13<05:15,  2.71it/s, loss=0.323]  

Error loading image /kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized/Thumbs.db. Skipping.


Epoch [15/15]: 100%|██████████| 1378/1378 [08:30<00:00,  2.70it/s, loss=0.0179] 


Epoch [15/15], Average Loss: 0.1230
Test Accuracy: 96.08%
Test Precision: 95.22%
Test Recall: 97.08%
Test F1 Score: 96.14%
Confusion Matrix:
[[2604  135]
 [  81 2692]]


_______________________________________